In [3]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import numpy as np
output_notebook()

Loading BokehJS ...

In [5]:
def kalmN(N, druk = False):
    
    """
    Predykcja
        X - przewidywana wartosc stanu
        P - kowariancja stanu
        A n x n - macierz przejscia stanu
        B n x 1 - macierz sterowania
    Korekcja
        H n x 1 - macierz wizazaca stany z wielkosciami mierzonymi
        R - macierz kowariancji szumu pomirowego
        Q - macierz kowariancji procesu
        Y - pomiar w kroku k
        V - measurement residual w kroku k
        S - kowariancja predykcji pomiaru w kroku k
        K - wzmocnienie
    
    
    """
    
    
    
    dt = 0.1#krok czasowy
    
    #model
    X = np.array([[0.0],[0.0],[0.1],[0.1]])    
    P = np.diag((0.1, 0.1, 0.1, 0.1))
    A = np.array([[1, 0 , dt, 0],\
                 [0, 1, 0, dt],\
                 [0, 0, 1, 0],\
                 [0, 0, 0, 1]])
    Q = np.eye((X.shape)[0])# X is Nx1
    B = np.eye((X.shape)[0])# X is Nx1
    U = np.zeros(((X.shape)[0],1))# U is Nx1
       
    H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
    Y = np.array([[X[0,0] + np.abs(np.random.randn())],\
                  [X[1,0] + np.abs(np.random.randn())]])
    
    R = np.eye((Y.shape)[0])# X is Nx1   
    
    if druk == True:
        print("X", X.shape)
        print("P", P.shape)
        print("A", A.shape)
        print("Q", Q.shape)
        print("B", B.shape)
        print("U", U.shape)
        print("Y", Y.shape)
        print("H", H.shape)
        print("R", R.shape)    
    kf_out = []
    y_out = []
    for i in np.arange(0,N):
        #pomiar
        Y = np.array([[X[0,0] + np.abs(0.1*np.random.randn())], \
                  [X[1,0] + np.abs(0.1*np.random.randn())]])
    
        #predykcja
        X = np.dot(A,X) + np.dot(B,U)# X[k] = A*X[k-1] + B * u[k-1] 
        P = np.dot(A, np.dot(P,A.T)) + Q# P[k] = A*P[k-1]A' + Q
        #korekcja
        #HX = np.dot(H,X)
        V = Y - np.dot(H,X)# measurement residual w kroku k
        S = np.dot(H, np.dot(P,H.T)) + R# predykcja w kroku k kowariancji szumu pomiarowego
        K = np.dot(P, np.dot(H.T, np.linalg.inv(S)))
        X = X + np.dot(K, V)
        P = P - np.dot(K, np.dot(S, K.T))  
        kf_out.append(X[0][0])#wyjscie KF
        y_out.append(Y[0][0])#pomiar
    
    return kf_out, y_out

kf_out, y_out = kalmN(30)
t = np.linspace(0,10,30)
p1 = figure(plot_width = 650, plot_height=350)
p1.line(t,kf_out, legend="kf_out", color="red", line_width=2)
p1.line(t,y_out, legend="y_out", color="blue", line_width=2)
show(p1)